In [4]:
pconnGrad = '/scratch/a/arisvoin/edickie/SPINS_glasser_tian/group-HCPS1200_atlas-GlasserTian_desc-subcorticalS2_conn.pconn.nii'

In [9]:
import nibabel as nib
from brainspace.gradient import GradientMaps
import pandas as pd


'''create_gradient takes in a nii file type, and the number of gradients desired as variables 'nifty' and 'n' respectively
   Returns a matrix of the number of elements / regions of interest x the number of gradients as a pandas dataframe''' 


def create_gradient(nifty, n):
    # load the nii with nibabel
    conn = nib.load(nifty)
    # create matrix with float data
    matrix = conn.get_fdata()
    # load GradientMaps
    gm = GradientMaps(n_components=n, random_state=0)
    # assert that the number of gradients requested does not exceed the regions of interest 
    max_grads = matrix.shape[0]
    assert n <= max_grads, 'Number of gradients exceeds rows in the matrix'
    # create the gradient as a matrix
    gm.fit(matrix)
    gradient = gm.gradients_
    # convert gradient to a pandas dataframe
    grad_df = pd.DataFrame(data = gradient, 
    columns=[f'grad{num + 1}' for num in range(gm.gradients_.shape[1])])
    return grad_df

grad_df = create_gradient(pconnGrad, 1)
grad_df

/home/a/arisvoin/lbassman/.conda/envs/nilearn_brainspace/lib/python3.8/site-packages/brainspace/gradient/embedding.py:70: UserWarning: Affinity is not symmetric. Making symmetric.
  warnings.warn('Affinity is not symmetric. Making symmetric.')


,grad1
0,-0.436793
1,-0.085681
2,0.023754
3,-0.241134
4,0.399192
...,...
387,-0.992800
388,-1.196641
389,-0.053247
390,-0.569917


In [11]:
from pathlib import Path

def write_df_column_to_pscalar_nib(df, pscalar_template, 
        to_filename = None, columns = None, labelname_column = None):
    ''' write a column from a pandas dataframe to pscalar file in nibabel 
     Parameters
    ----------
    df: pandas DataFrame
        Pandas dataframe containing data to write to file (with rows as parcels)
    
    pscalar_template: nibabel.Cifti2Image or filepath
        A pscalar.nii or pterseries.nii file or image to read the parcel axis from
    
    to_filename: str or path
        (optional) path (with extension .pscalar.nii) to write the output to
    columns: str or list
        (optional) list of columns to select for the pscalar image
    labelnames_column: str
        (optional) name of column that contains labelnames that should match the pscalar parcel names
        If this is given than data will be merged/aligned with the parcel axis before writing file.
    
    Returns
    -------
    pscalar_img: nibabel.Cifti2Image
        nibabel image of the pscalar output
        '''
           ## read the parcel axis from a template file
    if isinstance(pscalar_template, (str, Path)):
        pscalar_template = nib.load(pscalar_template)
    template_axis = pscalar_template.header.get_index_map(1)
    axis1_parcels = nib.cifti2.cifti2_axes.ParcelsAxis.from_index_mapping(template_axis)
    assert (isinstance(axis1_parcels, nib.cifti2.cifti2_axes.ParcelsAxis)), "Did not creat parcel axis"

    if labelname_column:
        axis_df = pd.DataFrame({'labelnames': axis1_parcels.name})
        df = df.merge(axis_df, 
            left_on = labelname_column, right_on = 'labelnames', how = 'right')
    
    ## grab columns if they exist
    if columns:
        df = df[columns]

    # grab the data column(s) and resphape them to the shape for pscalar 
    ## note that the dataobj shape should be (n_map, n_parcels)
    data_vals = df.transpose().values
    if len(data_vals.shape) == 1:
        data_vals = np.reshape(data_vals, (1,data_vals.shape[0]))
    
    ## assert that the number of parcels matches the length of the data array
    assert (axis1_parcels.size == len(df.index)), 'The number of parcels does not match the number of rows'

    ## create a scalar axis with names inlcuding the column names
    axis0_scalars = nib.cifti2.cifti2_axes.ScalarAxis(name = df.columns)
    
    ## combine all the bits together
    new_header = nib.Cifti2Header.from_axes((axis0_scalars, axis1_parcels))
    pscalar_img = nib.Cifti2Image(
        dataobj=data_vals, header = new_header)
    if to_filename:
        nib.cifti2.save(
            img = pscalar_img,filename=to_filename)

    return(pscalar_img)

In [18]:
grad_image = write_df_column_to_pscalar_nib(grad_df, pconnGrad)

grad_image